# **Sweep Notebook**
This notebook is easily customizable for easier sweeping and finetuning rather than having messy dev sweeps that all look different. This will be the go to for sweeping.

## **Import Libraries**

In [ ]:
import wandb
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import numpy as np


# --- Global Black-Scholes parameters and grids ---
r = 0.05          # Risk-free rate
sigma = 0.2       # Volatility
K = 100           # Strike price
T = 1.0           # Time to maturity (in years)
S_max = 250       # Max stock price in spatial domain
S_min = 0         # Min stock price
N = 500           # Number of spatial grid points

# Create spatial grid
S = torch.linspace(S_min, S_max, N).view(-1, 1).requires_grad_()
t = torch.linspace(0, T, N).view(-1, 1).requires_grad_()

## **Model**

In [ ]:
class blackScholesPINN(nn.Module):
    def __init__(self, input_dim=2, output_dim=1, hidden_layers=4, neurons_per_layer=64, activation_fn=nn.Tanh()):
        super().__init__()
        layers = []

        layers.append(nn.Linear(input_dim, neurons_per_layer))
        layers.append(activation_fn)

        for _ in range(hidden_layers - 1):
            layers.append(nn.Linear(neurons_per_layer, neurons_per_layer))
            layers.append(activation_fn)

        layers.append(nn.Linear(neurons_per_layer, output_dim))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)


def initialize_weights(model, method='xavier'):
    for m in model.modules():
        if isinstance(m, nn.Linear):
            if method == 'xavier':
                nn.init.xavier_uniform_(m.weight)
            elif method == 'kaiming':
                nn.init.kaiming_uniform_(m.weight)
            elif method == 'normal':
                nn.init.normal_(m.weight, mean=0.0, std=0.1)
            if m.bias is not None:
                nn.init.zeros_(m.bias)

## **Losses**

In [ ]:
def pde_loss(model, S, t):
    S.requires_grad_(True)
    t.requires_grad_(True)
    X = torch.cat((S, t), dim=1)
    V = model(X)

    V_t = torch.autograd.grad(V, t, grad_outputs=torch.ones_like(V), create_graph=True)[0]
    V_S = torch.autograd.grad(V, S, grad_outputs=torch.ones_like(V), create_graph=True)[0]
    V_SS = torch.autograd.grad(V_S, S, grad_outputs=torch.ones_like(V_S), create_graph=True)[0]

    residual = V_t + 0.5 * sigma**2 * S**2 * V_SS + r * S * V_S - r * V
    return torch.mean(residual.pow(2))


def boundary_loss(model, t):
    S0 = torch.zeros_like(t)
    S_high = torch.full_like(t, S_max)

    bc_low = model(torch.cat((S0, t), dim=1))
    bc_high = model(torch.cat((S_high, t), dim=1))
    expected_high = S_max - K * torch.exp(-r * (T - t))

    return torch.mean(bc_low.pow(2)) + torch.mean((bc_high - expected_high).pow(2))


def initial_loss(model, S):
    t0 = torch.zeros_like(S)
    X0 = torch.cat((S, t0), dim=1)

    V_pred = model(X0)
    V_true = torch.clamp(S - K, min=0.0)

    return torch.mean((V_pred - V_true).pow(2))

## **Configuration**

In [ ]:
epochs = 7500
num_sweeps = 20

config_flags = {
    'sweep_hidden_layers': False,
    'sweep_neurons': False,
    'sweep_activation': False,
    'sweep_init': False,
    'sweep_learning_rate': False,
    'sweep_loss_weights': True,
    'sweep_method': 'bayes'  # Choose from: 'random', 'bayes', 'grid'
}

# --------------------------
# DEFAULT VALUES (used if not swept)
# --------------------------
fixed_defaults = {
    'hidden_layers': 6,
    'neurons_per_layer': 32,
    'activation': 'relu',
    'init_method': 'xavier',
    'initial_lr': 0.005,
    'pde_weight_scale': 7.6,
    'bc_weight_scale': 1.35,
    'ic_weight_scale': 0.05
}

# --------------------------
# DYNAMIC SWEEP CONFIG BUILDER
# --------------------------
def generate_sweep_config(flags, defaults):
    sweep_config = {
        'method': flags['sweep_method'],
        'metric': {
            'name': 'final_total_loss',
            'goal': 'minimize'
        },
        'parameters': {}
    }

    if flags['sweep_hidden_layers']:
        sweep_config['parameters']['hidden_layers'] = {
            'values': [2, 4, 6, 8]
        }
    else:
        sweep_config['parameters']['hidden_layers'] = {
            'value': defaults['hidden_layers']
        }

    if flags['sweep_neurons']:
        sweep_config['parameters']['neurons_per_layer'] = {
            'values': [32, 64, 128]
        }
    else:
        sweep_config['parameters']['neurons_per_layer'] = {
            'value': defaults['neurons_per_layer']
        }

    if flags['sweep_activation']:
        sweep_config['parameters']['activation'] = {
            'values': ['tanh', 'relu', 'silu']
        }
    else:
        sweep_config['parameters']['activation'] = {
            'value': defaults['activation']
        }

    if flags['sweep_init']:
        sweep_config['parameters']['init_method'] = {
            'values': ['xavier', 'kaiming', 'normal']
        }
    else:
        sweep_config['parameters']['init_method'] = {
            'value': defaults['init_method']
        }

    if flags['sweep_learning_rate']:
        sweep_config['parameters']['initial_lr'] = {
            'min': 0.0001,
            'max': 0.01,
            'distribution': 'log_uniform_values'
        }
    else:
        sweep_config['parameters']['initial_lr'] = {
            'value': defaults['initial_lr']
        }

    if flags['sweep_loss_weights']:
        sweep_config['parameters']['pde_weight_scale'] = {
            'min': 5.0, 'max': 15.0, 'distribution': 'log_uniform_values'
        }
        sweep_config['parameters']['bc_weight_scale'] = {
            'min': 0.1, 'max': 3.5, 'distribution': 'log_uniform_values'
        }
        sweep_config['parameters']['ic_weight_scale'] = {
            'min': 0.05, 'max': 3.5, 'distribution': 'log_uniform_values'
        }
    else:
        sweep_config['parameters']['pde_weight_scale'] = {'value': defaults['pde_weight_scale']}
        sweep_config['parameters']['bc_weight_scale'] = {'value': defaults['bc_weight_scale']}
        sweep_config['parameters']['ic_weight_scale'] = {'value': defaults['ic_weight_scale']}

    return sweep_config

## **Sweep Function**

In [28]:
def train_pinn_sweep():
    wandb.init()
    config = wandb.config

    # Set up activation
    activation_map = {
        'tanh': nn.Tanh(),
        'relu': nn.ReLU(),
        'silu': nn.SiLU()
    }
    activation_fn = activation_map.get(config.activation, nn.Tanh())

    # Build model
    model = blackScholesPINN(
        hidden_layers=config.hidden_layers,
        neurons_per_layer=config.neurons_per_layer,
        activation_fn=activation_fn
    )

    initialize_weights(model, config.init_method)

    # Fixed hyperparameters
    lr = config.initial_lr
    pde_weight = config.pde_weight_scale
    bc_weight = config.bc_weight_scale
    ic_weight = config.ic_weight_scale

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=300)

    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()

        V = model(torch.cat((S, t), dim=1))  # assumes S and t are defined globally or imported

        pde_l = pde_loss(model, S, t)
        bc_l = boundary_loss(model, t)
        ic_l = initial_loss(model, S)

        total_loss = pde_weight * pde_l + bc_weight * bc_l + ic_weight * ic_l
        total_loss.backward()
        optimizer.step()
        scheduler.step(total_loss)

        wandb.log({
            "epoch": epoch,
            "total_loss": total_loss.item(),
            "pde_loss": pde_l.item(),
            "bc_loss": bc_l.item(),
            "ic_loss": ic_l.item(),
            "learning_rate": optimizer.param_groups[0]['lr']
        })

        if epoch % 500 == 0:
            print(f"[{epoch}] Total: {total_loss.item():.5f}, PDE: {pde_l.item():.5f}, BC: {bc_l.item():.5f}, IC: {ic_l.item():.5f}")

    wandb.log({
        "final_total_loss": total_loss.item()
    })
    wandb.finish()

## **Instantiate Sweep**

In [29]:
sweep_config = generate_sweep_config(config_flags, fixed_defaults)
sweep_id = wandb.sweep(sweep_config, project="Black-Scholes-Architecture-Sweep")

print(f"Sweep ID: {sweep_id}")
print("wandb agent " + sweep_id)
print(f"wandb.agent('{sweep_id}', function=train_pinn_sweep, count={num_sweeps})")

Create sweep with ID: ie5y2esn
Sweep URL: https://wandb.ai/tobiassafie-drexel-university/Black-Scholes-Architecture-Sweep/sweeps/ie5y2esn
Sweep ID: ie5y2esn
wandb agent ie5y2esn
wandb.agent('ie5y2esn', function=train_pinn_sweep, count=20)


### **Paste Function Call**

In [30]:
wandb.agent('ie5y2esn', function=train_pinn_sweep, count=20)

wandb: Agent Starting Run: 8iqztt4m with config:
wandb: 	activation: relu
wandb: 	bc_weight_scale: 0.6813706299260899
wandb: 	hidden_layers: 6
wandb: 	ic_weight_scale: 0.06114557165119223
wandb: 	init_method: xavier
wandb: 	initial_lr: 0.005
wandb: 	neurons_per_layer: 32
wandb: 	pde_weight_scale: 7.892454965443275


[0] Total: 13219.44043, PDE: 0.00949, BC: 19082.99023, IC: 3545.25781
[500] Total: 46.92128, PDE: 0.00094, BC: 1.52816, IC: 750.21936
[1000] Total: 48.22533, PDE: 0.00212, BC: 2.30302, IC: 762.76031
[1500] Total: 19.03658, PDE: 0.00151, BC: 0.93409, IC: 300.72888
[2000] Total: 0.48572, PDE: 0.00168, BC: 0.18460, IC: 5.66961
[2500] Total: 0.97751, PDE: 0.00150, BC: 1.18764, IC: 2.55815
[3000] Total: 0.27542, PDE: 0.00133, BC: 0.04997, IC: 3.77611
[3500] Total: 0.87833, PDE: 0.00111, BC: 1.02301, IC: 2.82102
[4000] Total: 0.40666, PDE: 0.00108, BC: 0.16489, IC: 4.67346
[4500] Total: 0.36508, PDE: 0.00090, BC: 0.04864, IC: 5.31184
[5000] Total: 0.22413, PDE: 0.00097, BC: 0.01417, IC: 3.38261
[5500] Total: 0.21624, PDE: 0.00091, BC: 0.01398, IC: 3.26368
[6000] Total: 0.20752, PDE: 0.00085, BC: 0.01339, IC: 3.13525
[6500] Total: 1.40245, PDE: 0.00079, BC: 1.88967, IC: 1.77764
[7000] Total: 0.21081, PDE: 0.00071, BC: 0.03905, IC: 2.92136


bc_loss,██▇▆▆▆▅▄▃▂▁▁▁▁▁▂▂▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▃
epoch,▁▁▁▁▁▂▂▂▂▂▂▂▃▄▄▄▄▄▄▄▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇█████
final_total_loss,▁
ic_loss,▇▅▇███▆▆▅▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
learning_rate,█████▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
pde_loss,▁▁▂▂▆█▅▆▇▆▅▅▅▅▅▅▅▅▅▅▄▄▄▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂
total_loss,█▇▇▇▆▅▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
bc_loss,0.01209
epoch,7499
final_total_loss,0.28247
ic_loss,4.41505


wandb: Agent Starting Run: bddtzxrv with config:
wandb: 	activation: relu
wandb: 	bc_weight_scale: 0.17432197256539922
wandb: 	hidden_layers: 6
wandb: 	ic_weight_scale: 0.3743040365636722
wandb: 	init_method: xavier
wandb: 	initial_lr: 0.005
wandb: 	neurons_per_layer: 32
wandb: 	pde_weight_scale: 9.366002206217289


[0] Total: 5064.65527, PDE: 0.00003, BC: 20659.70508, IC: 3909.16064
[500] Total: 14.73156, PDE: 0.03975, BC: 8.67912, IC: 34.32058
[1000] Total: 2.56679, PDE: 0.02486, BC: 0.04652, IC: 6.21391
[1500] Total: 0.75609, PDE: 0.00764, BC: 0.90098, IC: 1.40910
[2000] Total: 0.67091, PDE: 0.00327, BC: 0.81204, IC: 1.33239
[2500] Total: 1.43748, PDE: 0.00585, BC: 0.54219, IC: 3.44145
[3000] Total: 0.61003, PDE: 0.00373, BC: 0.32967, IC: 1.38295
[3500] Total: 3.25212, PDE: 0.00334, BC: 14.43818, IC: 1.88065
[4000] Total: 0.57364, PDE: 0.00162, BC: 0.58429, IC: 1.21990
[4500] Total: 0.61079, PDE: 0.00753, BC: 0.55775, IC: 1.18361
[5000] Total: 0.56923, PDE: 0.00394, BC: 0.54368, IC: 1.16901
[5500] Total: 0.57302, PDE: 0.00392, BC: 0.55061, IC: 1.17651
[6000] Total: 0.56837, PDE: 0.00378, BC: 0.54550, IC: 1.16979
[6500] Total: 0.56718, PDE: 0.00332, BC: 0.55113, IC: 1.17557
[7000] Total: 0.56585, PDE: 0.00319, BC: 0.55096, IC: 1.17542


...

[0] Total: 7179.45801, PDE: 0.00360, BC: 16822.02148, IC: 3045.44824
[500] Total: 44.27391, PDE: 0.03910, BC: 28.45803, IC: 372.50339
[1000] Total: 1.11510, PDE: 0.00261, BC: 0.26250, IC: 11.28290
[1500] Total: 0.90459, PDE: 0.00042, BC: 1.24959, IC: 4.47646
[2000] Total: 0.21104, PDE: 0.00038, BC: 0.03120, IC: 2.24353
[2500] Total: 0.57095, PDE: 0.00022, BC: 1.07908, IC: 1.44365
[3000] Total: 0.17133, PDE: 0.00022, BC: 0.01892, IC: 1.86673
[3500] Total: 0.16287, PDE: 0.00021, BC: 0.01672, IC: 1.78061
[4000] Total: 0.41930, PDE: 0.00021, BC: 0.73537, IC: 1.32502
[4500] Total: 0.17640, PDE: 0.00211, BC: 0.01303, IC: 1.63560
[5000] Total: 0.14747, PDE: 0.00015, BC: 0.01320, IC: 1.62722
[5500] Total: 0.23928, PDE: 0.00013, BC: 0.29995, IC: 1.32740
[6000] Total: 0.14020, PDE: 0.00014, BC: 0.01159, IC: 1.55208
[6500] Total: 0.13679, PDE: 0.00013, BC: 0.01115, IC: 1.51758
[7000] Total: 0.17944, PDE: 0.00051, BC: 0.04549, IC: 1.78473


bc_loss,▆█▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▁▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▅▅▅▆▆▆▇▇▇▇▇▇▇████
final_total_loss,▁
ic_loss,▆█▆▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
learning_rate,█████▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
pde_loss,▃▂▄▂█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
total_loss,█▇▇▆▆▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
bc_loss,0.01819
epoch,7499
final_total_loss,0.13702
ic_loss,1.49146


wandb: Agent Starting Run: r2gn8dyv with config:
wandb: 	activation: relu
wandb: 	bc_weight_scale: 0.10626312945567588
wandb: 	hidden_layers: 6
wandb: 	ic_weight_scale: 1.1514937653144417
wandb: 	init_method: xavier
wandb: 	initial_lr: 0.005
wandb: 	neurons_per_layer: 32
wandb: 	pde_weight_scale: 8.485220514084965


[0] Total: 8730.85547, PDE: 0.00005, BC: 26030.88672, IC: 5179.99463
[500] Total: 3.72725, PDE: 0.00435, BC: 27.77281, IC: 0.64189
[1000] Total: 1.49914, PDE: 0.00156, BC: 8.66118, IC: 0.49111
[1500] Total: 1.31220, PDE: 0.00134, BC: 6.81860, IC: 0.50047
[2000] Total: 1.27369, PDE: 0.00144, BC: 6.58171, IC: 0.48815
[2500] Total: 1.22669, PDE: 0.00140, BC: 6.06977, IC: 0.49486
[3000] Total: 1.19128, PDE: 0.00140, BC: 5.68782, IC: 0.49933
[3500] Total: 1.20151, PDE: 0.00141, BC: 6.07365, IC: 0.47252
[4000] Total: 7.25894, PDE: 0.00128, BC: 39.58966, IC: 2.64109
[4500] Total: 1.27379, PDE: 0.00124, BC: 6.49821, IC: 0.49736
[5000] Total: 1.25040, PDE: 0.00126, BC: 6.24858, IC: 0.49997
[5500] Total: 1.24261, PDE: 0.00131, BC: 6.16323, IC: 0.50074
[6000] Total: 1.23973, PDE: 0.00128, BC: 6.12872, IC: 0.50162
[6500] Total: 1.23851, PDE: 0.00128, BC: 6.11632, IC: 0.50169
[7000] Total: 1.23800, PDE: 0.00128, BC: 6.11155, IC: 0.50171


bc_loss,▇█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▂▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇████
final_total_loss,▁
ic_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
learning_rate,███████▄▄▄▄▄▄▄▄▃▃▃▃▃▃▃▃▃▃▃▃▁▁▁▁▁▁▁▁▁▁▁▁▁
pde_loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
total_loss,▂▅▁▁▁▁▁▁▁▁▁█▁▁▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
bc_loss,6.11052
epoch,7499
final_total_loss,1.23787
ic_loss,0.50167
